<a href="https://colab.research.google.com/github/derek-shing/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/module1-Intro-to-Neural-Networks/LS_DS_431_Intro_to_NN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Neural Networks Assignment

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
### Hidden Layer:
### Output Layer:
### Neuron:
### Weight:
### Activation Function:
### Node Map:
### Perceptron:


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [0]:
##### Your Code Here #####
import numpy as np
np.random.seed(1)



inputs = np.array([
    [0, 0, 1],
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1]
])

correct_outputs = [[1], [0], [1], [1]]

In [0]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
  return sigmoid(x) * (1 - sigmoid(x))

In [4]:
weights = 2 * np.random.random((3, 1)) - 1
weights

array([[-0.16595599],
       [ 0.44064899],
       [-0.99977125]])

In [5]:
weighted_sum = np.dot(inputs, weights)
weighted_sum

array([[-0.99977125],
       [-0.72507825],
       [-1.16572724],
       [-0.55912226]])

In [6]:
activated_output = sigmoid(weighted_sum)
activated_output

array([[0.2689864 ],
       [0.3262757 ],
       [0.23762817],
       [0.36375058]])

In [7]:
error = correct_outputs - activated_output
error

array([[ 0.7310136 ],
       [-0.3262757 ],
       [ 0.76237183],
       [ 0.63624942]])

In [8]:
adjustments = error * sigmoid_derivative(activated_output)
adjustments

array([[ 0.17948714],
       [-0.079436  ],
       [ 0.18792752],
       [ 0.15391469]])

In [9]:
weights += np.dot(inputs.T, adjustments)
weights

array([[-0.05746447],
       [ 0.51512768],
       [-0.55787791]])

In [10]:
for iteration in range(10000):
  # Weighted sum of inputs/weights
  weighted_sum = np.dot(inputs, weights)
  
  # Activate!
  activated_output = sigmoid(weighted_sum)
  
  # Calculate error
  error = correct_outputs - activated_output
  
  # Calculate weight adjustments - watch linked videos for more intuition!
  adjustments = error * sigmoid(activated_output)
  
  # Update weights
  weights += np.dot(inputs.T, adjustments)
  
print('Weights after training')
print(weights)

print('Output after training')
print(activated_output)

Weights after training
[[-13.85204949]
 [-13.85204949]
 [ 21.13616677]]
Output after training
[[1.        ]
 [0.00140287]
 [0.99931405]
 [0.99931405]]


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [0]:
##### Your Code Here #####

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?